In [ ]:
import pandas as pd
from utils.import_utils import *
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from src.unit_proccessing import  *
from src.utils.stats_utils import *
import plotly.express as px
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pyod.models.ecod import ECOD
from pythresh.thresholds.hist import HIST
from pythresh.thresholds.hist import HIST
from pythresh.thresholds.filter import FILTER

In [ ]:
with initialize(config_path='../configuration', version_base='1.1'):
    config = compose(config_name='main.yaml')

In [ ]:
features_class = UnitDataProcessing(config)

In [ ]:
df_unit = features_class.df_unit
self = features_class
feature_name = 'f__total_duration'
df = df_unit[~pd.isnull(df_unit[feature_name])].copy()
df[feature_name] = round(df[feature_name]/(3600/6))*10#/df_unit[~pd.isnull(df_unit[feature_name])]['f__number_answered']

## Plot the distribution of answer time set

In [ ]:
df[feature_name].hist(bins=48)

## USE ECOD algorithm that makes use of cumulative function and is non-parametric for detecting anomalies in answer time set

In [ ]:
model = ECOD(contamination=0.11)#FILTER(method='medfilt', random_state=42))
model.fit(df[[feature_name]])

In [ ]:
from pythresh.thresholds.dsn import DSN
from pythresh.thresholds.comb import COMB

from pythresh.thresholds.yj import YJ
decision_scores = model.decision_scores_  # raw outlier scores on the train data

# get outlier labels
#thres = HIST()
#thres = DSN()
thres = FILTER(method='medfilt', random_state=42)
labels = thres.eval(decision_scores)
thres.thresh_

In [ ]:
df['anomaly'] = model.predict(df[[feature_name]])
df['anomaly'].value_counts(), df['anomaly'].value_counts()/df['anomaly'].count()

In [ ]:
df[df['anomaly']==0][feature_name].min(), df[df['anomaly']==0][feature_name].max()

In [ ]:
(0.011*3600*24)/60

In [ ]:
df['anomaly'].value_counts()

In [ ]:
data_true = df[df['anomaly']==0][feature_name]
data_false = df[df['anomaly']==1][feature_name]

# Determine the bin edges based on the entire dataset
bins = np.histogram_bin_edges(df[feature_name], bins=48)


plt.hist(data_true, bins=bins, alpha=0.5, color='blue', label='True')
plt.hist(data_false, bins=bins, alpha=0.5, color='red', label='False')
plt.show()

In [ ]:
sns.boxplot(df,y=feature_name, x='anomaly')
plt.show()